# Monitor the active tasks on Google Earth Engine

Run this notebook to monitor download tasks in progress on Google Earth Engine.

In [1]:
# Necessary imports
import os
import json
from tqdm.notebook import tqdm
import random
import pandas as pd
import time
from db_utils import DB
from dotenv import load_dotenv

## Load environment and project details

As with the other notebooks, we load credentials and project details from a hidden ```.env``` file.

In [2]:
# Load environment variables (including path to credentials) from '.env' file
env_file_path = "../.env"

assert load_dotenv(dotenv_path=env_file_path) == True, "[ERR] Failed to load environment!"
assert "GOOGLE_APPLICATION_CREDENTIALS" in os.environ, "[ERR] Missing $GOOGLE_APPLICATION_CREDENTIAL!"
assert "GS_USER_PROJECT" in os.environ, "[ERR] Missing $GS_USER_PROJECT!"
key_file_path = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
assert os.path.exists(key_file_path), f"[ERR] Google credential key file does not exist: \n{key_file_path} "
assert "ML4FLOODS_BASE_DIR" in os.environ, "[ERR] Missing $ML4FLOODS_BASE_DIR!"
base_path = os.environ["ML4FLOODS_BASE_DIR"]
assert os.path.exists(base_path), f"[ERR] Base path does not exist: \n{base_path} "
print("[INFO] Successfully loaded FloodMapper environment.")

[INFO] Successfully loaded FloodMapper environment.


**Set the path to the JSON task file here**

In [3]:
# Path to the JSON file outout by download script
json_path = "../scripts/2023-03-13_13.45.27.json"
os.path.exists(json_path)

True

## Display a progress bar

The cells here can be quickly run in sequence to produce progress bars for the tasks being tracked by the database. Note that the ```01_download_images.py``` script must remain running for this notebook to work. 

In [4]:
# Connect to the database
db_conn = DB()

[INFO] Connecting to DB 'floodmapper-db'.
[INFO] Connection successfully established.


In [5]:
# Load the JSON 
task_list = json.load(open(json_path, "r"))
n_tasks = len(task_list)
print(f"JSON currently contains {n_tasks} task entries.")

# Convert to a DataFrame
tasks_df = pd.DataFrame(task_list)
tasks_df["gridname"] = tasks_df["description"].str.split("_").str[0]

JSON currently contains 133 task entries.


In [ ]:
# Query the DB for the download status of the images.
image_ids = tuple(tasks_df['description'].unique())
if len(image_ids) > 1:
    db_query = """SELECT image_id, in_progress 
                  FROM images_download 
                  WHERE image_id IN {};""".format(image_ids)
else:
    db_query = """SELECT image_id, in_progress 
                  FROM images_download 
                  WHERE image_id = '{}';""".format(image_ids[0])
image_db = db_conn.run_query(db_query, fetch = True)
print("Returned {:d} rows from the DB".format(len(image_db)))

In [10]:
# Query the DB for the download status of the images
image_ids = tuple(tasks_df['description'].unique())
query = (f"SELECT image_id, in_progress "
         f"FROM images_download "
         f"WHERE image_id IN %s;")
data = (tuple(image_ids),)
image_db = db_conn.run_query(query, data, fetch=True)

In [12]:
# Initialise progress bar for all available tasks. 
batch_bar = tqdm(total=len(tasks_df), 
                 dynamic_ncols=True, 
                 leave=False, 
                 position=0, 
                 desc="All Tasks",
                 colour="GREEN")

# Add a progress bar for every unique grid patch present in task list
grid_bars = {}
for name, gdf in tasks_df.groupby(by='gridname'):
    grid_bars[name] = tqdm(total=len(gdf), 
                           dynamic_ncols=True, 
                           leave=True, 
                           position=0, 
                           desc=name)

# Logic : Check all tasks, keep removing them as and when the 
# in_progress flag is set to 0 for the task in the database.
while len(tasks_df) >= 1:
    
    # Loop through the tasks grouped by gridname
    for name, gdf in tasks_df.groupby(by='gridname'):
        for i, task in gdf.iterrows():
            
            # Check if download is still marked as in-progress in the DB
            desc = task['description']
            ip = image_db[image_db['image_id'] == desc]['in_progress'].item()
            
            # Do nothing if still in-progress
            if ip == 1:
                continue

            ## Drop entry if not still in-progress
            if ip == 0:
                tasks_df.drop(i, inplace = True)
                grid_bars[name].update()
                batch_bar.update()
        
        time.sleep(0.25)

All Tasks:   0%|                                                                                              …

GRID11716:   0%|                                                                                              …

GRID11717:   0%|                                                                                              …

GRID11885:   0%|                                                                                              …

GRID11886:   0%|                                                                                              …

...

GRID12225:   0%|                                                                                              …

GRID12394:   0%|                                                                                              …

GRID12562:   0%|                                                                                              …

GRID12563:   0%|                                                                                              …

KeyboardInterrupt: 